# Kernel Semantik

Dalam contoh kode ini, Anda akan menggunakan kerangka kerja AI [Kernel Semantik](https://aka.ms/ai-agents-beginners/semantic-kernel) untuk membuat agen dasar.

Tujuan dari contoh ini adalah untuk menunjukkan langkah-langkah yang nantinya akan kita gunakan dalam contoh kode tambahan saat mengimplementasikan berbagai pola agen.


## Impor Paket Python yang Dibutuhkan


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Membuat Klien

Dalam contoh ini, kita akan menggunakan [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) untuk mengakses LLM.

`ai_model_id` didefinisikan sebagai `gpt-4o-mini`. Cobalah mengganti model ini dengan model lain yang tersedia di marketplace GitHub Models untuk melihat hasil yang berbeda.

Agar kita dapat menggunakan `Azure Inference SDK` yang digunakan untuk `base_url` pada GitHub Models, kita akan menggunakan konektor `OpenAIChatCompletion` dalam Semantic Kernel. Ada juga [konektor lain yang tersedia](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) untuk menggunakan Semantic Kernel dengan penyedia model lainnya.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Membuat Agen

Di bawah ini kita membuat Agen bernama `TravelAgent`.

Untuk contoh ini, kita menggunakan instruksi yang sangat sederhana. Anda dapat mengubah instruksi ini untuk melihat bagaimana agen merespons secara berbeda.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Menjalankan Agen

Sekarang kita dapat menjalankan Agen dengan mendefinisikan thread bertipe `ChatHistoryAgentThread`. Pesan sistem yang diperlukan diberikan ke argumen kata kunci `messages` pada `invoke_stream` agen.

Setelah ini didefinisikan, kita membuat `user_inputs` yang akan menjadi apa yang dikirimkan pengguna ke agen. Dalam contoh ini, kita menetapkan pesan ini menjadi `Plan me a sunny vacation`.

Silakan ubah pesan ini untuk melihat bagaimana agen merespons secara berbeda.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
